**Random Forest Classification**

In [66]:
import numpy as np
import pandas as pd

In [75]:
file = './processedData.csv'
data = pd.read_csv(file)

In [83]:
convert_feature = ['cDNA_position','CDS_position','Protein_position']
data['cDNA_position'] = data['cDNA_position'].astype(str).str.replace(r'\D+', '')
data['CDS_position'] = data['CDS_position'].astype(str).str.replace(r'\D+', '')
data['Protein_position'] = data['Protein_position'].astype(str).str.replace(r'\D+', '')
for i in range(len(convert_feature)):
  data[convert_feature[i]] = data[convert_feature[i]].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [84]:
features = data.drop("CLASS" ,axis= 1)
labels = np.array(data['CLASS'])
feature_list = list(features.columns)
features = np.array(features)

In [85]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size = 0.25, random_state = 42)

In [86]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (38060, 54)
Training Labels Shape: (38060,)
Testing Features Shape: (12687, 54)
Testing Labels Shape: (12687,)


In [87]:
print(data.columns)

Index(['Unnamed: 0', 'CHROM', 'POS', 'REF', 'ALT', 'AF_ESP', 'AF_EXAC',
       'AF_TGP', 'CLNVC', 'CLASS', 'IMPACT', 'SYMBOL', 'cDNA_position',
       'CDS_position', 'Protein_position', 'STRAND', 'SIFT', 'PolyPhen',
       'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'Amino_acid_ref', 'Amino_acid_alt',
       'unknown', 'germline', 'somatic', 'inherited', 'paternal', 'maternal',
       'de-novo', 'biparental', 'uniparental', 'not-tested',
       'tested-inconclusive', 'other_origin', 'missense_variant',
       'synonymous_variant', 'other', 'IE', 'IELOCATION',
       'has_Hereditary_cancer-predisposing_syndrome',
       'has_Hereditary_breast_and_ovarian_cancer_syndrome',
       'has_Familial_cancer_of_breast', 'has_Dilated_cardiomyopathy_1G',
       'has_Limb-girdle_muscular_dystrophy,_type_2J',
       'has_Cardiovascular_phenotype', 'has_Hypertrophic_cardiomyopathy',
       'has_Ataxia-telangiectasia_syndrome',
       'has_Hereditary_nonpolyposis_colon_cancer',
       'has_Dilated_Cardiomyo

In [91]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [93]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_labels, predictions))

Accuracy: 0.7875778355797273


In [96]:
cm = metrics.confusion_matrix(test_labels, predictions)
print("Consusion Matrix ", cm)

Consusion Matrix  [[8992  547]
 [2148 1000]]


In [97]:
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
# Showing the confusion matrix
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,8992,547
Actual Positive,2148,1000


In [98]:
# Creating a function to report confusion metrics
def confusion_metrics (conf_matrix):
# save confusion matrix and slice into four pieces
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)
    
    # calculate accuracy
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1- conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    print('-'*50)
    print(f'Accuracy: {round(conf_accuracy,2)}') 
    print(f'Mis-Classification: {round(conf_misclassification,2)}') 
    print(f'Sensitivity: {round(conf_sensitivity,2)}') 
    print(f'Specificity: {round(conf_specificity,2)}') 
    print(f'Precision: {round(conf_precision,2)}')
    print(f'f_1 Score: {round(conf_f1,2)}')

In [100]:
confusion_metrics(cm)

True Positives: 1000
True Negatives: 8992
False Positives: 547
False Negatives: 2148
--------------------------------------------------
Accuracy: 0.79
Mis-Classification: 0.21
Sensitivity: 0.32
Specificity: 0.94
Precision: 0.94
f_1 Score: 0.48
